## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [13]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SBBBSBSS'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
652


In [6]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_1'
Query="SELECT %s FROM weather"%Features
pdf = sqlContext.sql(Query).toPandas()

In [7]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_1')

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [12]:
def MS(Mat):
    return np.nanmean(Mat**2)

def MS_Coeff(coeff):
    Features='station, year, %s'%coeff
    Query="SELECT %s FROM weather"%Features
    pdf = sqlContext.sql(Query).toPandas()
    year_station_table=pdf.pivot(index='year', columns='station', values=coeff)

    mean_by_year=np.nanmean(year_station_table,axis=1)
    mean_by_station=np.nanmean(year_station_table,axis=0)
    tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
    tbl_minus_station = year_station_table-mean_by_station
    
    r0=RMS(year_station_table)
    r1=RMS(tbl_minus_station)
    r2=RMS(tbl_minus_year)
    print 'total MS                   = ',r0
    print 'MS removing mean-by-station= ',r1, ', fraction explained= ',1-r1*1.0/r0
    print 'MS removing mean-by-year   = ',r2, ', fraction explained= ',1-r2*1.0/r0
    
MS_Coeff('coeff_1')    
MS_Coeff('coeff_2')    
MS_Coeff('coeff_3')    
MS_Coeff('coeff_4')   

total MS                   =  3089242.65143
MS removing mean-by-station=  831509.149582 fraction explained=  0.730837216948
MS removing mean-by-year   =  2282087.38995 fraction explained=  0.26127933366
total MS                   =  572855.74951
MS removing mean-by-station=  334599.677293 fraction explained=  0.415909367097
MS removing mean-by-year   =  308346.042514 fraction explained=  0.461738766212
total MS                   =  281719.821376
MS removing mean-by-station=  249701.164801 fraction explained=  0.113654255558
MS removing mean-by-year   =  119346.449046 fraction explained=  0.57636474259
total MS                   =  233475.366452
MS removing mean-by-station=  216367.293934 fraction explained=  0.073275706889
MS removing mean-by-year   =  94764.0688572 fraction explained=  0.594115343742


In [10]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 3089242.65143
0 after removing mean by year    = 2282087.38995
0 after removing mean by stations= 476547.26866
1 after removing mean by year    = 357741.972267
1 after removing mean by stations= 335277.766808
2 after removing mean by year    = 327791.291559
2 after removing mean by stations= 324736.721778
3 after removing mean by year    = 323352.391803
3 after removing mean by stations= 322676.65332
4 after removing mean by year    = 322328.053025
4 after removing mean by stations= 322140.841996
